<a href="https://colab.research.google.com/github/Thomasche69/deepseek-code-generation-finetuned/blob/main/deepseek_finetuned_code_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install git+https://github.com/k4black/codebleu.git
!pip install sophia-opt
!pip install tree-sitter-python

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-g8xlt_j3
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-g8xlt_j3
  Resolved https://github.com/unslothai/unsloth.git to commit 038e6d4c8d40207a87297ab3aaf787c19b1006d1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.1.8-py3-none-any.whl size=174982 sha256=79cfd65dd8834e6120026c5d01d810c6bea5df5a43e3f06081e00d98f38655aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-nd70328n/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.1.8
    Uninstalling unsloth-2025.1.8:
      Successfully uninstalled unsloth-2025.1.8
  Cloning https://github.com/k4black/codebleu.git to /tmp/pip-req-build-zamuli41
 

In [ ]:
from huggingface_hub import notebook_login
notebook_login()  # Login to Hugging Face

In [ ]:
from datasets import load_dataset
dataset_train = load_dataset("LimYeri/LeetCode_Python_Solutions_v2", split = "train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/587 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15734 [00:00<?, ? examples/s]

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = True,
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


0it [00:00, ?it/s]

🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/231 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.9k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.1.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt =  """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Solve the following LeetCode problem in Python.

### Input_
Problem title: {}
Problem content: {}

### Response:
Code content: {}"""

EOS_TOKEN = tokenizer.eos_token  # Ensure tokenizer is defined

def formatting_prompt_func(examples):
    """
    Efficiently formats examples into instruction-response text for fine-tuning.
    """
    # Extract lists
    titles = examples.get("title", [])
    slugs = examples.get("question_content", [])
    codes= examples.get("content", [])


    # Ensure all lists have the same length
    assert len(titles) == len(slugs) == len(codes), "Mismatched input lengths!"

    # Use list comprehension for efficiency
    texts = [alpaca_prompt.format(title,slug,code) + EOS_TOKEN for title,slug,code in zip(titles,slugs, codes)]

    return {"text": texts}

# Apply formatting function to dataset
dataset_train = dataset_train.map(formatting_prompt_func, batched=True)

Map:   0%|          | 0/15734 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from trl import SFTTrainer
from sophia_opt import SophiaG

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset_train,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=4,  # Increase parallel processing speed
    packing=False,  # Keep False unless using long sequences
    args=TrainingArguments(
        per_device_train_batch_size=4,  # Keep batch size the same
        gradient_accumulation_steps=8,  # Reduce accumulation for stability
        warmup_ratio=0.1,  # ✅ Use ratio instead of fixed steps (6% of total training)
        num_train_epochs=3,  # ✅ Train for 15 full passes over dataset
        learning_rate=1.5e-5,  # Lower LR for stability
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_strategy="epoch",  # ✅ Logs only at the end of each epoch
        optim="adamw_8bit",
        weight_decay=0.03,  # Improve generalization
        lr_scheduler_type="cosine_with_restarts",  # Better decay schedule
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

optimizer = SophiaG(
    trainer.model.parameters(),
    lr=2e-5,   # Recommended learning rate range
    betas=(0.965, 0.99),  # Recommended for LLM fine-tuning
    rho=0.02,  # Controls second-order approximation
    weight_decay=0.01
)

trainer.optimizer = optimizer

trainer.train()


Map (num_proc=4):   0%|          | 0/15734 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 15,734 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 8
\        /    Total batch size = 32 | Total steps = 1,473
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
492,0.706800
984,0.376600


Step,Training Loss
492,0.706800
984,0.376600
1473,0.319100


TrainOutput(global_step=1473, training_loss=0.4677838068468103, metrics={'train_runtime': 17597.5298, 'train_samples_per_second': 2.682, 'train_steps_per_second': 0.084, 'total_flos': 2.521930081700512e+18, 'train_loss': 0.4677838068468103, 'epoch': 2.994407727503813})

In [ ]:
notebook_login()  # Login to Hugging Face


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import HfApi

model_push_path = "Thomasche/deepseek-leetcode"

# Push model to Hugging Face
model.push_to_hub(model_push_path)
tokenizer.push_to_hub(model_push_path)

print(f"Model uploaded to Hugging Face: https://huggingface.co/{model_push_path}")

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Thomasche/deepseek-leetcode


README.md:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Model uploaded to Hugging Face: https://huggingface.co/Thomasche/deepseek-leetcode
